![Alt text](http://www.ucm.es/logo/ucm.png "a title")

<div align="center"> 
<font size=5> Máster en Nuevas Tecnologías Electrónicas y Fotónicas </font>
</div>
    
<div align="center"> 
<font size=4> Óptica Digital, curso 2022-2023 </font>
</div>

    
<div align="center"> 
<font size=4>   </font>
</div>


<div align="center"> 
<font size=5> Ejercicio 3 - Micro-óptica </font>
</div>

- **Fecha**: 
        
- **Alumno**: Alex Recuenco (Alejandro G Recuenco)

In [1]:
from matplotlib import rcParams

rcParams["figure.dpi"] = 250



# Introducción
En estos ejercicios nos vamos a centrar en elementos microópticos como lentes de Fresnel y redes de difraccion.

# Enunciados

1. Sea una lente estándar de focal f′=10 mm y diámetro D=0.5 mm.

 a. Determine, en el marco XY y utilizando la aproximación de elemento delgado, la distribución de intensidad en el plano focal, mediante el modelo de Rayleigh-Sommerfeld.  Determine, a partir de esta imagen, el perfil radial de intensidad.
 
 b. Haga lo mismo para una lente de Fresnel binaria de amplitud con los mismos parámetros constructivos.
 
 c. Idem para una lente de Fresnel de fase, de desfase $\pi$.
    
 d. Compare los 3 perfiles de intensidad.
 
 Para la visualización de los focos, haga un recorte de la imagen (plt.xlim y plt.ylim,  o cut_resample) para poder observarlo adecuadamente. También puede utilizar el parámetro logarithm de .draw() para resaltar los anillos.

2. Sea una lente de Fresnel binaria de fase, cuya focal es f′=10 mm y diámetro D=0.5 mm. Determine, en el marco XY y utilizando la aproximación de elemento delgado y el modelo de Rayleigh-Sommerfeld, la intensidad máxima en el foco en función del desfase entre las almenas de la lente, dentro del rango (0 - 2π).



3. Sea una red de Ronchi (con factor de relleno 0.5). Se pretende visualizar el efecto Talbot, mediante el formalismo XZ. Para ello se ilumina la red con una onda plana y se propaga mediante la función RS una distancia de varias veces la distancia de Talbot: $z_T = 2 p^2 / \lambda$, siendo p el periodo y $\lambda$ la longitud de onda (elegid los parámetros adecuados) para una correcta visualización. Como existe un efecto de borde al propagar, se puede recortar en el eje x, para eliminar dicho efecto de borde.

Proponga un valor para los parámetros que no vengan en el enunciado. En todos los casos, los resultados son gráficas de intensidad y fase luminosa.

## Resolución 1


In [ ]:
from dataclasses import dataclass
from typing import Tuple
from diffractio.scalar_sources_XY import Scalar_source_XY
from diffractio.scalar_masks_XY import Scalar_mask_XY

from diffractio import degrees, mm, plt, sp, um, np
from diffractio.scalar_sources_X import Scalar_source_X
from diffractio.scalar_masks_X import Scalar_mask_X


@dataclass
class SimulationConfig:
    simulation_radius: float = 0.3 * mm
    simulation_resolution: int = 1024
    amplification: int = 1


@dataclass
class SimulationParams:
    focal: float = 10 * um
    lens_diameter: float = 0.5 * mm

    observation_distance: float = 2 * mm


def plot_evolution(params=SimulationParams(), config=SimulationConfig(), title=None):

    x0 = np.linspace(
        -config.simulation_radius,
        config.simulation_radius,
        config.simulation_resolution,
    )
    y0 = np.linspace(
        -config.simulation_radius,
        config.simulation_radius,
        config.simulation_resolution,
    )
    u1 = Scalar_source_XY(x=x0, y=y0, wavelength=params.wavelength)
    u1.plane_wave(A=1, theta=np.pi / 2, phi=0)

    circle_mask = Scalar_mask_XY(x=x0, y=y0, wavelength=params.wavelength)
    circle_mask.circle(r0=params.circle_pos, radius=params.circle_radius)
    square_mask = Scalar_mask_XY(x=x0, y=y0, wavelength=params.wavelength)
    square_mask.square(
        r0=params.square_pos, size=params.square_size, angle=params.square_angle
    )
    total_mask = circle_mask + square_mask
    # Cast into 1s and zeros... I don't know how else to do this
    total_mask.u = (total_mask.u > 0) + 0
    total_mask.draw()
    plt.show()
    u2 = u1 * total_mask

    observation_plane = u2.RS(
        params.observation_distance,
        new_field=True,
        amplification=(config.amplification, config.amplification),
        verbose=True,
    )
    draw_intensity_phase_XY(
        observation_plane,
        title=title
        or f"Wave at {params.observation_distance/mm:.1f}mm of source. Observation window of size {config.amplification*config.simulation_radius/mm:.1f}mm",
    )


# Original problem
plot_evolution(
    params=SimulationParams(
        observation_distance=2 * mm, circle_radius=25 * um, square_size=100 * um
    ),
)
plot_evolution(
    params=SimulationParams(
        observation_distance=4 * mm, circle_radius=25 * um, square_size=100 * um
    ),
)

# Modified problem with more overlap

plot_evolution()
plot_evolution(
    params=SimulationParams(observation_distance=4 * mm),
)
plot_evolution(
    params=SimulationParams(observation_distance=4 * mm, circle_pos=(0, 0)),
)


### **a.** Lente estándar en aproximación TEA

### b. Lente binaria de amplitud

### c. Lente binaria de fase

### d. Comparación

## Resolución 2

## Resolución 3